In [1]:
import pdfplumber
import pytesseract
import cv2
from PIL import Image
import numpy as np

## Paths

In [2]:
from pathlib import Path

image_path = Path("../testfiles")

#image_file = image_path / "Tom_Thumb_test_receipt.jpg"

## Helper Functions

In [3]:
def is_scanned_pdf(pdf_path):
    """ Does not

    :param pdf_path: _description_
    :type pdf_path: _type_
    :return: _description_
    :rtype: _type_
    """
    with pdfplumber.open(pdf_path) as pdf:
        first_page = pdf.pages[0]
        if first_page.extract_text() is None:
            return True
        return False
    
def extract_text_from_pdf(pdf_path):
    """ Imported from original code
    
    Not Used yet

    :param pdf_path: _description_
    :type pdf_path: _type_
    :return: _description_
    :rtype: _type_
    """
    text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            image = page.to_image()
            page_text = pytesseract.image_to_string(image.original)
            text += page_text
    return text

def preprocess_image(image_path):
    """Preprocess the image for better OCR results.

    :param image_path: _description_
    :type image_path: _type_
    :return: _description_
    :rtype: _type_
    """
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    #gray.show()
    ##_, binary = cv2.threshold(image, 127, 255, cv2.THRESH_BINARY)
    # Deskewing can be added here with more complex algorithms
    
    # Dilation makes text bolder
    kernel = np.ones((2, 2), np.uint8)
    dilated = cv2.dilate(gray, kernel, iterations=1)    
    return gray

# Load the image and process 

Preprocess and load into a *large* text area

1. Create the list of test files
2. Loop through the list
3. Load the images and preprocess them
4. Create the sentences list
5. Seach the sentances

In [4]:
test_files = [
    "Market_Street_test_receipt.jpg",
    "Tom_Thumb_test_receipt.jpg",
    "Trek_Allen_test Receipt.jpg",
    "Walmart_test_receipt.png"
]

In [9]:
import re

for file in test_files:
    img = preprocess_image(image_path / file)
    #cv2.imshow("Processed Image", img)
    #cv2.waitKey(0)
    #cv2.destroyAllWindows()
    #continue
    
    # Perform OCR using pytesseract
    text = pytesseract.image_to_string(img)

    # Print the size of the extracted text buffer
    print(f"Loaded {len(text)} characters from {file}")

    sentences = re.split(r'\n+', text.strip())
    print(sentences)
    print(f"found {len(sentences)} sentences in {file}")
    date = extract_dates(sentences)
    if date:
        print(f"Found date(s) in {file}: {date}")
    print(f"======\n")    

Loaded 586 characters from Market_Street_test_receipt.jpg
['rl f', 'BOARS HEAD TURKE 6.66 F', '2UB TOLAL $6, 66', 'fOTAL 0.8', 'CASH $7.00', 'CHANGE $0.14', 'Tota! number of in ; » seld 2', 'HOW was your shop ing exper rience?', 'We Value Tali feedback!', 'oo the Gh code below far a 2 winute survey.', "di ik you for _suop'ng at Market Street", 'r help with you Rewards account ar', 'online shopp ing call 1-855- fie 15 a', 'Have a complatet, compliment, ¢', 'om cal -877-BAUNTTE', 'Oy visit waw.marketstreetunited.com', 'Cashier :0037 Nan gS ue Ca', 'wrsday, Auigu st 14, 2¢ O25 U3:38 PM', 'fore :563 POS:032 Transact ion:6123', 'ane 172) 906-3830', '385 W. Bethany Dr. Allen, TX 75013', 'request?']
found 22 sentences in Market_Street_test_receipt.jpg
no dates found

Loaded 837 characters from Tom_Thumb_test_receipt.jpg
['Store 1788 Dir Arthur Sanchez', 'Main:€972) 527-0961 Rx:¢972) 527-0970', '7801 Alma Drive', 'PLANO TX', 'OS', '0017880520081 2508111332', 'GROCERY Price', '7084789185  HONST

In [ ]:
for s in sentences:
    print(f"Sentence: {s} | Length: {len(s)}")

## Analyze the extracted text

1. Look for a date and time

In [8]:
import datetime
def extract_dates(sentences):
    for s in sentences:
        if matches := re.findall(r'\d{1,2}[-/]\d{1,2}[-/]\d{2,4} \d{2}:\d{2}:\d{2}', s):
            print("1")
            format_string = '%m/%d/%y %H:%M:%S'
            break
        elif matches := re.findall(r'\d{1,2}[-/]\d{1,2}[-/]\d{2,4} \d{1,2}:\d{2} [AP]M', s):
            print("2")
            format_string = '%m/%d/%Y %I:%M %p'
            break
        elif matches := re.findall(r'\d{1,2}[-/]\d{1,2}[-/]\d{2,4} \d{1,2}:\d{2}', s):
            print("3")
            format_string = '%m/%d/%y %H:%M'
            break
        else:
            continue
        
    if matches:
        print(f"Found date(s) in sentence: {s} | Matches: {matches}")
        try:
            date = [datetime.datetime.strptime(date, format_string) for date in matches]
            return date
        except ValueError as e:
            print(f"Error parsing dates: {e}")
            return None
        
    else:
        print(f"no dates found")
        return None

In [ ]:
import re
    
s = 'Sam'
matches = re.findall(r'\b(Monday|Tuesday|Wednesday|Thursday|Friday|Saturday|Sunday)\b', s)
print(matches)

['S', 'a']
